In [1]:
import requests
from bs4 import BeautifulSoup
import re
from selenium import webdriver
import urllib3
import math
import time
import pandas as pd
import xlsxwriter

In [34]:
writer = pd.ExcelWriter("workbook3.xlsx", engine='xlsxwriter')

In [25]:
url = "https://www.ratemyprofessors.com/search.jsp?queryoption=HEADER&queryBy=teacherName&schoolName=University+of+Utah&schoolID1606&"

In [20]:
#Just the last few pages.
#url = "https://www.ratemyprofessors.com/search.jsp?query=&queryoption=HEADER&stateselect=&country=&dept=&queryBy=teacherName&facetSearch=&schoolName=University+of+Utah&offset=2860&max=20"

In [5]:
# result = requests.get(url)
# src = result.content
# soup = BeautifulSoup(src, 'html.parser')
# professors = soup.find_all(class_="PROFESSOR")

In [26]:
options = webdriver.ChromeOptions()
options.add_argument("--test-type")
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
#options.add_argument('--headless')
driver = webdriver.Chrome('chromedriver.exe', options=options)
driver.get(url)

In [27]:
accept_cookies = driver.find_elements_by_class_name("close-this")
accept_cookies[0].click()

In [28]:
professors = []
nextLink = driver.find_elements_by_class_name("nextLink")
professors_list = []
while nextLink:
    page = driver.page_source
    soup = BeautifulSoup(page, 'html.parser')
    professors = soup.find_all(class_="PROFESSOR")
    url = "https://www.ratemyprofessors.com/"
    for prof in professors:
        extension = re.search(r"ShowRatings\.jsp\?tid=\d+", str(prof.find("a")))
        name = re.search(r"\"main\">(.+)</span>", str(prof))
        university_and_dept = re.search(r"\"sub\">(.+)</span>", str(prof))
        dep = university_and_dept[1].split(",")
        professor = {'name': name[1], 'url': url+extension[0], 'department': dep[1]}
        professors_list.append(professor)
    nextLink[0].click()
    nextLink = driver.find_elements_by_class_name("nextLink")

[{'name': 'Rudolph, Sid ', 'url': 'https://www.ratemyprofessors.com/ShowRatings.jsp?tid=16773', 'department': ' Physics'}, {'name': 'Baruche, Amanda ', 'url': 'https://www.ratemyprofessors.com/ShowRatings.jsp?tid=52254', 'department': ' Social Science'}, {'name': 'Muller, Sheila ', 'url': 'https://www.ratemyprofessors.com/ShowRatings.jsp?tid=93487', 'department': ' Fine Arts'}, {'name': 'Toscano, Margaret ', 'url': 'https://www.ratemyprofessors.com/ShowRatings.jsp?tid=93489', 'department': ' Languages'}, {'name': 'Darling, Ann ', 'url': 'https://www.ratemyprofessors.com/ShowRatings.jsp?tid=93493', 'department': ' Communication'}, {'name': 'Jorgensen, Erik ', 'url': 'https://www.ratemyprofessors.com/ShowRatings.jsp?tid=111426', 'department': ' Biology'}, {'name': 'Skliar, Mikhail ', 'url': 'https://www.ratemyprofessors.com/ShowRatings.jsp?tid=131612', 'department': ' Engineering'}, {'name': 'Ring, Terry ', 'url': 'https://www.ratemyprofessors.com/ShowRatings.jsp?tid=131707', 'department

In [31]:
# For running just a few professors.

# index_list = [0, 1, 2]
# professors_list = [professors_list[i] for i in index_list]
# print(professors_list)

[{'name': 'Rudolph, Sid ', 'url': 'https://www.ratemyprofessors.com/ShowRatings.jsp?tid=16773', 'department': ' Physics'}, {'name': 'Baruche, Amanda ', 'url': 'https://www.ratemyprofessors.com/ShowRatings.jsp?tid=52254', 'department': ' Social Science'}, {'name': 'Muller, Sheila ', 'url': 'https://www.ratemyprofessors.com/ShowRatings.jsp?tid=93487', 'department': ' Fine Arts'}]


In [35]:
for prof in professors_list:
    prof_url = prof['url']
    #Start Web Driver for the given professor
    options = webdriver.ChromeOptions()
    options.add_argument("--test-type")
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--incognito')
    driver = webdriver.Chrome('chromedriver.exe', options=options)
    driver.get(prof_url)
    #Show all comments
    myList = driver.find_element_by_class_name("react-tabs__tab-panel")
    myButton = myList.find_elements_by_tag_name("button")
    while len(myButton) > 0:
        myButton[0].click()
        myList = driver.find_element_by_class_name("react-tabs__tab-panel")
        myButton = myList.find_elements_by_tag_name("button")
    #Get all the Quality ratings
    ratings = driver.find_elements_by_class_name("kUfTQq")
    quality_ratings = []
    for rating in ratings:
        quality_ratings.append(rating.text)
    print(len(quality_ratings))
    #Get all the Difficulty Ratings
    ratings = driver.find_elements_by_class_name("cKZySD")
    difficulty_ratings = []
    for rating in ratings:
        difficulty_ratings.append(rating.text)
    print(len(difficulty_ratings))
    #Get all the Comments
    all_comments = driver.find_elements_by_class_name("edLmhN")
    comments = []
    for comment in all_comments:
        comments.append(comment.text)
    print(len(comments))
    #Get all Course Numbers
    all_course_numbers = driver.find_elements_by_class_name("hBbYdP")
    course_numbers = []
    for course_number in all_course_numbers:
        course_numbers.append(course_number.text)
    while "" in course_numbers:
        course_numbers.remove("")
    print(course_numbers)
    data = pd.DataFrame(list(zip(course_numbers, comments, quality_ratings, difficulty_ratings)), 
                    columns=['Course Number', 'Comments', 'Quality Rating', 'Difficulty Rating'])
    data.to_excel(writer, sheet_name=prof['name'])

6
17
17
['PHY2010', 'PHYS2010', 'PHYS2010', 'PHYS2020', 'PHYS2010', 'PHYSICS', 'PHYS2010', 'ACCESS', 'PHYSICS2010', 'ACCESS', 'PHYSICS2010', 'PHY2010', 'PHYSICS', 'PHYCS2020', 'PHYS2020', 'PHYSC2020', 'PHYSICS20']
1
2
2
['SW6412', 'SEXIOLOGY101']
8
22
22
['ARTH4260', 'ARTH3250', 'ARTH3200', 'ARTH3200', 'ARTH4800', 'ARTH3250', 'ARTH2500', 'ARTH4210', 'ARTH4260', 'ARTH2500', 'HIST3500', 'ART2400', 'ARTHISTROY', 'ARTHISTORY', 'ART2500', 'ARTH', 'ARTHISTORY2500', 'ARTHISTORY', 'ARTH2200', 'ARTH2500', 'ARTH2500', 'ARTH']


In [36]:
writer.save()

In [9]:
# # Eventually will loop through each of the urls that were found.
# test_url2 = "https://www.ratemyprofessors.com/ShowRatings.jsp?tid=523219"

In [10]:
# result = requests.get(test_url2)
# src = result.content
# soup = BeautifulSoup(src, 'html.parser')

In [11]:
# options = webdriver.ChromeOptions()
# options.add_argument("--test-type")
# options.add_argument('--ignore-certificate-errors')
# options.add_argument('--incognito')
# #options.add_argument('--headless')
# # driver = webdriver.Chrome('chromedriver.exe', options=options)
# driver.get(test_url2)

In [15]:
# myList = driver.find_element_by_class_name("react-tabs__tab-panel")
# myButton = myList.find_elements_by_tag_name("button")
# while len(myButton) > 0:
#     myButton[0].click()
#     myList = driver.find_element_by_class_name("react-tabs__tab-panel")
#     myButton = myList.find_elements_by_tag_name("button")

<selenium.webdriver.remote.webelement.WebElement (session="ea1938d70b6f669880c29f42e8142699", element="852a3389-c537-4bfd-9430-52be40ebb4eb")>
<selenium.webdriver.remote.webelement.WebElement (session="ea1938d70b6f669880c29f42e8142699", element="9bc7bc88-623b-46d8-99bf-147e05b2e6e4")>
<selenium.webdriver.remote.webelement.WebElement (session="ea1938d70b6f669880c29f42e8142699", element="56f13e4a-d0e6-42a7-a095-be8a9b53509a")>
<selenium.webdriver.remote.webelement.WebElement (session="ea1938d70b6f669880c29f42e8142699", element="d66f0ca9-06ce-47ab-baeb-d8e635e0eb17")>
<selenium.webdriver.remote.webelement.WebElement (session="ea1938d70b6f669880c29f42e8142699", element="812461fe-2e6e-44bb-b05e-16f85f72912e")>
<selenium.webdriver.remote.webelement.WebElement (session="ea1938d70b6f669880c29f42e8142699", element="7985d3ab-026d-4333-af7e-0598f066769f")>
<selenium.webdriver.remote.webelement.WebElement (session="ea1938d70b6f669880c29f42e8142699", element="e33b4909-15f7-4194-82c5-3c2e03e4a25d")>

In [30]:
# #Get all the Quality ratings
# ratings = driver.find_elements_by_class_name("kUfTQq")
# quality_ratings = []
# for rating in ratings:
#     quality_ratings.append(rating.text)
# print(len(quality_ratings))

149


In [29]:
# #Get all the Difficulty Ratings
# ratings = driver.find_elements_by_class_name("cKZySD")
# difficulty_ratings = []
# for rating in ratings:
#     difficulty_ratings.append(rating.text)
# print(len(difficulty_ratings))

149


In [33]:
# #Get all the Comments
# all_comments = driver.find_elements_by_class_name("edLmhN")
# comments = []
# for comment in all_comments:
#     comments.append(comment.text)
# print(len(comments))

149


In [39]:
# #Get all Course Numbers
# all_course_numbers = driver.find_elements_by_class_name("hBbYdP")
# course_numbers = []
# for course_number in all_course_numbers:
#     course_numbers.append(course_number.text)
# while "" in course_numbers:
#     course_numbers.remove("")
# print(course_numbers)

['312R', '312R', 'MUSIC312R', 'MUS312R', 'MUS314R', '314R', 'MUS312', 'MUS314R', '314R', '312R', '312R', '314R', 'MUSIC312R', 'MUSIC312R', 'MUS314R', 'MUS312', 'MUSIC312R', 'MUSIC312R', 'MUSIC312R', 'MUSIC312R', 'MUSIC312R', 'MUSIC312R', 'MUS312R', '312R', 'MUS312R', '312R', '312R', 'MUS314R', '312R', 'MUS312R', '312R', 'MUSIC312R', '312R', 'MUS312R', 'MUS312R', 'MUSIC312R', '312R', 'MUSIC312R', 'MUSIC312R', 'MUSIC312R', 'MUSIC312R', 'MUSIC312R', 'MUS312R', 'MUS384R', '312', 'MUSIC 312R', 'MUSIC', 'MUSIC 312-R', 'MUSIC 312R', 'MUSIC 312R', 'MUSIC 312R', 'MUSIC 312R', 'MUSIC 312R', 'MUSIC 312R', 'MATH 312R', 'MUS312R', 'MUSIC 312R', 'MUSIC 312R', 'MUSIC 213R', 'MUSIC312R', 'MUSIC 312R', 'MENS CHORUS', 'MUSIC 312R', '312R', 'MUSIC312R', 'MUSIC 312R', 'MUSIC312R', 'MUSIC 312', 'MUSIC 312R', 'MUSIC312R', 'MUSIC 312R', 'MUSIC 312R', 'MUS322', 'MUSIC 312R', 'MUS312', 'MUSIC 312R', 'MUSIC 219', 'MUSIC312R', 'MUSIC312', 'MUSIC235B', 'MUSIC314R', 'MUS312R', 'MUSIC312R', 'MUSIC321R', '312R', '31

In [40]:
# data = pd.DataFrame(list(zip(course_numbers, comments, quality_ratings, difficulty_ratings)), 
#                     columns=['Course Number', 'Comments', 'Quality Rating', 'Difficulty Rating'])
# print(data)

    Course Number                                           Comments  \
0            312R  Unfortunately this review won't matter much be...   
1            312R  In a word, this course is LIFE-CHANGING. Siste...   
2       MUSIC312R  She is an absolute Queen! I have never met a s...   
3         MUS312R  This is the best vocal teacher I have ever had...   
4         MUS314R  I've had the privilege of working with Sister ...   
..            ...                                                ...   
144    MENSCHORUS     Roz was one of the best teachers I've ever had   
145       MUS314R  Sister Hall is quite possibly one of the most ...   
146       MUS312R  Sister Hall is the greatest person in the worl...   
147       MUS314R  Absolutely incredible teacher - the way she te...   
148     MUSIC312R  Men's Chorus is the funnes class ever! Sis. Ha...   

    Quality Rating Difficulty Rating  
0              5.0               2.0  
1              5.0               3.0  
2              5.0

In [56]:
# name = driver.find_elements_by_class_name("cjgLEI")[0].text
# data.to_excel(writer, sheet_name=name)
# writer.save()